In [1]:

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pandas as pd
import cv2
import os
import imutils
from google.colab.patches import cv2_imshow
import pickle
import seaborn as sn
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import datetime
import platform

from matplotlib import pyplot as plt

In [2]:
# MTM : multiple template matching
from google.colab import drive
drive.mount("/F_distance")


Mounted at /F_distance


In [3]:
df = pd.read_csv("/F_distance/MyDrive/football_detection/dataset/5_minutes_.csv")
df

,Unnamed: 0,bbox,bbox_center,class_name
0,Frame_1,"(647, 508, 703, 564)","(675, 536)",football
1,Frame_2,"(633, 509, 692, 565)","(663, 537)",football
2,Frame_3,"(625, 505, 681, 561)","(653, 533)",football
3,Frame_4,"(618, 498, 674, 555)","(646, 526)",football
4,Frame_5,"(610, 495, 667, 551)","(639, 523)",football
...,...,...,...,...
7511,Frame_7512,NaN,NaN,NaN
7512,Frame_7513,NaN,NaN,NaN
7513,Frame_7514,NaN,NaN,NaN
7514,Frame_7515,NaN,NaN,NaN


In [5]:

df = pd.read_csv("/F_distance/MyDrive/football_detection/dataset/5_minutes_.csv")

# Remove rows with NaN values
df = df.dropna()

# Extract coordinates from the 'bbox_center' column
df['bbox_center'] = df['bbox_center'].apply(lambda x: tuple(map(int, x.strip('()').split(','))))
df['bbox_center']



0       (675, 536)
1       (663, 537)
2       (653, 533)
3       (646, 526)
4       (639, 523)
           ...    
7177    (109, 650)
7178     (83, 607)
7221    (806, 414)
7222    (777, 410)
7235    (243, 322)
Name: bbox_center, Length: 2466, dtype: object

In [9]:

df = pd.read_csv("/F_distance/MyDrive/football_detection/dataset/5_minutes_.csv")

# Remove rows with NaN values
df = df.dropna()
df


,Unnamed: 0,bbox,bbox_center,class_name
0,Frame_1,"(647, 508, 703, 564)","(675, 536)",football
1,Frame_2,"(633, 509, 692, 565)","(663, 537)",football
2,Frame_3,"(625, 505, 681, 561)","(653, 533)",football
3,Frame_4,"(618, 498, 674, 555)","(646, 526)",football
4,Frame_5,"(610, 495, 667, 551)","(639, 523)",football
...,...,...,...,...
7177,Frame_7178,"(67, 611, 152, 689)","(109, 650)",football
7178,Frame_7179,"(47, 571, 119, 642)","(83, 607)",football
7221,Frame_7222,"(777, 385, 834, 442)","(806, 414)",football
7222,Frame_7223,"(749, 384, 805, 435)","(777, 410)",football


In [ ]:
# Create the 'situation' column based on distance conditions
df['situation'] = pd.cut(df['distance'], bins=[-float('inf'), 10, 15, 100, float('inf')],
                         labels=['dribbled', 'kicked', 'camera', ''], right=False)


In [16]:

# Extract x and y coordinates from the 'bbox_center' column
df[['x', 'y']] = df['bbox_center'].str.strip('()').str.split(',', expand=True).astype(int)

# Calculate the Euclidean distance between consecutive rows
df['distance'] = df[['x', 'y']].diff().apply(
    lambda row: math.sqrt(row['x']**2 + row['y']**2) if not pd.isna(row['x']) else None,
    axis=1
)
df['situation'] = pd.cut(df['distance'], bins=[-float('inf'), 10, 100, 1000, float('inf')],
                         labels=['dribbled', 'kicked', 'camera', ''], right=False)

# Display the DataFrame with distances
df

,Unnamed: 0,bbox,bbox_center,class_name,x,y,distance,situation
0,Frame_1,"(647, 508, 703, 564)","(675, 536)",football,675,536,NaN,NaN
1,Frame_2,"(633, 509, 692, 565)","(663, 537)",football,663,537,12.041595,kicked
2,Frame_3,"(625, 505, 681, 561)","(653, 533)",football,653,533,10.770330,kicked
3,Frame_4,"(618, 498, 674, 555)","(646, 526)",football,646,526,9.899495,dribbled
4,Frame_5,"(610, 495, 667, 551)","(639, 523)",football,639,523,7.615773,dribbled
...,...,...,...,...,...,...,...,...
7177,Frame_7178,"(67, 611, 152, 689)","(109, 650)",football,109,650,54.561891,kicked
7178,Frame_7179,"(47, 571, 119, 642)","(83, 607)",football,83,607,50.249378,kicked
7221,Frame_7222,"(777, 385, 834, 442)","(806, 414)",football,806,414,748.316778,camera
7222,Frame_7223,"(749, 384, 805, 435)","(777, 410)",football,777,410,29.274562,kicked


In [14]:
df.to_csv('distance.csv', index=False)